<a href="https://colab.research.google.com/github/gabmachado97/jetson_segmentation/blob/main/conversor_fastai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Install packages


In [ ]:
!pip3 install onnx
!pip3 install onnxruntime

     |████████████████████████████████| 12.2MB 215kB/s 
     |████████████████████████████████| 4.1MB 12.2MB/s 


## 2. Convert fastai model to onnx

In [ ]:
import torch
import torch.onnx
from fastai.vision import *

#rtk model accuracy
def acc_rtk(input, target):
    target = target.squeeze(1)
    mask = target != 0
    return (input.argmax(dim=1)[mask]==target[mask]).float().mean()


path_pkl = '/content/drive/My Drive/Mestrado/Segmentação/'
path_weights = '/content/drive/My Drive/Mestrado/Segmentação/data/images/models/stage-2-weights'
learn = load_learner(path_pkl, 'export.pkl')
learn.load(path_weights)
model = learn.model

# Create the right input shape (e.g. for an image)
sample_batch_size = 1
channel = 3
height = 352
width = 288
dummy_input = torch.randn(sample_batch_size, channel, height, width).to('cuda')
torch_out = model(dummy_input)

torch.onnx.export(model, 
                  dummy_input, 
                  "/content/drive/My Drive/Mestrado/Segmentação/convert_onnx/rtk_plus_resnet34.onnx",
                   verbose=True,
                    export_params=True)

/usr/local/lib/python3.6/dist-packages/fastai/vision/models/unet.py:31: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if ssh != up_out.shape[-2:]:


graph(%input.1 : Float(1:304128, 3:101376, 352:288, 288:1, requires_grad=0, device=cuda:0),
      %layers.1.weight : Float(512:1, requires_grad=1, device=cuda:0),
      %layers.1.bias : Float(512:1, requires_grad=1, device=cuda:0),
      %layers.1.running_mean : Float(512:1, requires_grad=0, device=cuda:0),
      %layers.1.running_var : Float(512:1, requires_grad=0, device=cuda:0),
      %layers.3.0.0.weight : Float(1024:4608, 512:9, 3:3, 3:1, requires_grad=1, device=cuda:0),
      %layers.3.0.0.bias : Float(1024:1, requires_grad=1, device=cuda:0),
      %layers.3.1.0.weight : Float(512:9216, 1024:9, 3:3, 3:1, requires_grad=1, device=cuda:0),
      %layers.3.1.0.bias : Float(512:1, requires_grad=1, device=cuda:0),
      %layers.4.shuf.conv.0.weight : Float(1024:512, 512:1, 1:1, 1:1, requires_grad=1, device=cuda:0),
      %layers.4.shuf.conv.0.bias : Float(1024:1, requires_grad=1, device=cuda:0),
      %layers.4.bn.weight : Float(256:1, requires_grad=1, device=cuda:0),
      %layers.4.b

## 3. Optional

In [ ]:
import onnx

onnx_model = onnx.load("/content/drive/My Drive/Mestrado/Segmentação/convert_onnx/rtk_plus_resnet34.onnx")
onnx.checker.check_model(onnx_model)

In [ ]:
import onnxruntime
import torch

ort_session = onnxruntime.InferenceSession("/content/drive/My Drive/Mestrado/Segmentação/convert_onnx/rtk_resnet34.onnx")

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# compute ONNX Runtime output prediction
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(dummy_input)}
ort_outs = ort_session.run(None, ort_inputs)

# compare ONNX Runtime and PyTorch results
np.testing.assert_allclose(to_numpy(torch_out), ort_outs[0], rtol=1e-03, atol=1e-05)

print("Exported model has been tested with ONNXRuntime, and the result looks good!")

## Backup

In [ ]:
learn = load_learner('/content/drive/My Drive/Mestrado/Segmentação/', 'export.pkl')
learn.load('/content/drive/My Drive/Mestrado/Segmentação/data/images/models/stage-2-weights')
model = learn.model
model